# vector db 활용 학습
- 주제: @@@ 감성 분석 or 사용자 맞춤 데이터 출력
1. 적절한 크기의 데이터셋을 구한다.
2. 한국어 임베딩에 적절한 모델을 찾는다.
3. Chroma DB, FAISS, Pinecone 중 원하는 벡터 DB를 이용한다.

In [11]:
import pandas as pd
# import numpy as np

df = pd.read_csv('./data/train.tsv', sep='\t')

# print(f"원본 데이터 크기: {len(df)}")

# sample_10k = df.sample(n=10000, random_state=42)  # 재현성을 위한 random_state
sample_1k = df.sample(n=1000, random_state=42)

# # CSV 파일로 저장
# sample_10k.to_csv('./data/sentiment_10k_sample.csv', index=False)
sample_1k.to_csv('./data/train_1k.csv', index=False)

# print(f"10,000개 샘플 크기: {len(sample_10k)}")
# print(f"1,000개 샘플 크기: {len(sample_1k)}")

In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train_1k.csv')

df = df[['comments','hate']]

hate_mapping = {
    'none': 0,      # 혐오 표현 없음
    'offensive': 1, # 공격적인 표현 (약한 혐오)
    'hate': 2       # 명백한 혐오 표현
}

# 매핑 적용
df['hate'] = df['hate'].map(hate_mapping)

df.to_csv('./data/hate.csv', index=False)

In [13]:
import pandas as pd
import numpy as np
from pinecone import Pinecone, ServerlessSpec
import pinecone as pc
import os
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer
from collections import Counter

df = pd.read_csv('./data/hate.csv')

load_dotenv()
api_key = os.getenv('API_KEY')

pc = Pinecone(api_key=api_key)
index_name = 'reviewtext'

pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
        )
    )

KeyboardInterrupt: 

In [ ]:
try:
    # 기존 인덱스 확인
    indexes = pc.list_indexes()
    if index_name not in [idx.name for idx in indexes]:
        pc.create_index(
            name=index_name,
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
        print(f"인덱스 '{index_name}'를 생성했습니다.")
    else:
        print(f"인덱스 '{index_name}'가 이미 존재합니다.")
except Exception as e:
    print(f"인덱스 생성 중 오류 발생: {e}")

In [ ]:
# 인덱스 연결
idx = pc.Index(index_name)

# 임베딩 모델 로드 - 한국어에 좀 더 적합한 모델 사용
model = SentenceTransformer('jhgan/ko-sroberta-multitask')  # 한국어 특화 모델로 변경

# 데이터프레임 컬럼명 확인 및 처리
print("DataFrame 컬럼:", df.columns.tolist())

# 데이터 임베딩 및 업로드
vectors_to_upsert = []

In [ ]:
for i, row in df.iterrows():
    try:
        text = row['text']  # 리뷰 텍스트 컬럼명 확인 필요
        sentiment = row['sentiment']  # 감성 레이블 컬럼명 확인 필요
        
        # 고유 ID 생성 (기존 id 컬럼이 없다면)
        record_id = str(row.get('id', f"review_{i}"))
        
        # 텍스트 임베딩
        review_embed = model.encode(text).tolist()
        
        # 벡터 및 메타데이터 준비
        vector = (record_id, review_embed, {'text': text, 'sentiment': sentiment})
        vectors_to_upsert.append(vector)
        
        # 배치 처리 (100개씩)
        if len(vectors_to_upsert) >= 100:
            idx.upsert(vectors=vectors_to_upsert)
            print(f"{len(vectors_to_upsert)}개 벡터 업로드 완료")
            vectors_to_upsert = []
    
    except Exception as e:
        print(f"레코드 {i} 처리 중 오류: {e}")

# 남은 벡터 업로드
if vectors_to_upsert:
    idx.upsert(vectors=vectors_to_upsert)
    print(f"남은 {len(vectors_to_upsert)}개 벡터 업로드 완료")


In [ ]:

# 유사 리뷰 검색 및 감성 분석 함수
def analyze_sentiment(query_text, top_k=10):
    """
    입력된 쿼리와 유사한 리뷰를 찾고 종합적인 감성을 분석합니다.
    
    Args:
        query_text: 분석할 쿼리 텍스트
        top_k: 검색할 유사 리뷰 개수
        
    Returns:
        dict: 분석 결과 (유사 리뷰 목록, 긍정/부정 개수, 종합 감성)
    """
    # 쿼리 임베딩
    query_embed = model.encode(query_text).tolist()
    
    # 유사 리뷰 검색
    results = idx.query(
        vector=query_embed,
        top_k=top_k,
        include_metadata=True
    )
    
    # 결과 파싱
    matches = results.get('matches', [])
    reviews = []
    sentiments = []
    
    for match in matches:
        metadata = match.get('metadata', {})
        text = metadata.get('text', 'N/A')
        sentiment = metadata.get('sentiment', 'N/A')
        score = match.get('score', 0)
        
        reviews.append({
            'text': text,
            'sentiment': sentiment,
            'similarity_score': score
        })
        
        sentiments.append(sentiment)
    
    # 감성 분석
    sentiment_counts = Counter(sentiments)
    positive_count = sentiment_counts.get('positive', 0) + sentiment_counts.get('긍정', 0)
    negative_count = sentiment_counts.get('negative', 0) + sentiment_counts.get('부정', 0)
    
    # 종합 감성 판단
    if positive_count > negative_count:
        overall_sentiment = "긍정"
    elif negative_count > positive_count:
        overall_sentiment = "부정"
    else:
        overall_sentiment = "중립"
    
    # 결과 반환
    result = {
        'query': query_text,
        'similar_reviews': reviews,
        'sentiment_counts': {
            'positive': positive_count,
            'negative': negative_count
        },
        'overall_sentiment': overall_sentiment
    }
    
    return result

In [ ]:
# 사용자 입력 받기
if __name__ == "__main__":
    while True:
        query_text = input("\n원하는 문구를 입력하세요 (종료하려면 'q' 입력): ")
        
        if query_text.lower() == 'q':
            break
        
        # 감성 분석 실행
        result = analyze_sentiment(query_text, top_k=10)
        
        # 결과 출력
        print("\n===== 감성 분석 결과 =====")
        print(f"입력 쿼리: {result['query']}")
        print(f"총평: {result['overall_sentiment']} (긍정: {result['sentiment_counts']['positive']}, 부정: {result['sentiment_counts']['negative']})")
        
        print("\n----- 유사한 리뷰 목록 -----")
        for i, review in enumerate(result['similar_reviews'], 1):
            print(f"{i}. [{review['sentiment']}] {review['text'][:100]}... (유사도: {review['similarity_score']:.4f})")